<div style="text-align: center;">
    <center><img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ3Zwg8uptvuP3WYPy5qHUK1qllvZx5dxghGA&s" />
</div>

# <center>**Artificial Intelligence**
# <center>Connect 4 Project
## <center>AI Instructor
## <center>Eng. Yousef Elbaroudy
## <center>Team Members
### <center> سكشن 2
### <center> محمد امير ابوزيد احمد
### <center> محمد ايمن يحيى عبد السلام
###  <center> سكشن 3
### <center> محمد صبحي محمد عواد
### <center> يوسف هشام عبد الفتاح عبد العظيم
## <center>2025/2026

________________________________________________________________________________________

## <center> AI VS AI

### Importing Libraries and Defining Constants
We import `numpy` for the board representation, `random` for random choices, and `math` for infinity values.  
We also define constants for the board dimensions, player pieces, and window length.


In [6]:
import numpy as np
import random
import math

ROW_COUNT = 6
COLUMN_COUNT = 7

AI1 = 0
AI2 = 1

EMPTY = 0
AI1_PIECE = 1
AI2_PIECE = 2

WINDOW_LENGTH = 4


### Board Creation and Printing
- `create_board()` creates an empty board filled with zeros.
- `print_board()` flips the board vertically for a more natural view when printed.


In [7]:
def create_board():
    return np.zeros((ROW_COUNT, COLUMN_COUNT))

def print_board(board):
    print(np.flip(board, 0))


### Checking Valid Moves and Dropping Pieces
- `is_valid_location(board, col)` checks if the top row of a column is empty.
- `get_next_open_row(board, col)` returns the first empty row in a column.
- `drop_piece(board, row, col, piece)` places a piece on the board.


In [8]:
def is_valid_location(board, col):
    return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROW_COUNT):
        if board[r][col] == 0:
            return r

def drop_piece(board, row, col, piece):
    board[row][col] = piece


### Winning Move Detection
Checks horizontal, vertical, and diagonal sequences of 4 for a given piece.


In [9]:
def winning_move(board, piece):
    # Horizontal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT):
            if all(board[r][c+i] == piece for i in range(4)):
                return True
    # Vertical
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True
    # Positive diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(ROW_COUNT-3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True
    # Negative diagonal
    for c in range(COLUMN_COUNT-3):
        for r in range(3, ROW_COUNT):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False


### Scoring Mechanism for AI
- `evaluate_window(window, piece)` evaluates a window of 4 cells and gives a score based on AI advantage.
- `score_position(board, piece)` sums up scores horizontally, vertically, and diagonally to evaluate the whole board.


In [10]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = AI1_PIECE if piece == AI2_PIECE else AI2_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

def score_position(board, piece):
    score = 0
    center_array = [int(i) for i in list(board[:, COLUMN_COUNT//2])]
    score += center_array.count(piece) * 3

    # Horizontal
    for r in range(ROW_COUNT):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLUMN_COUNT-3):
            score += evaluate_window(row_array[c:c+WINDOW_LENGTH], piece)
    # Vertical
    for c in range(COLUMN_COUNT):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROW_COUNT-3):
            score += evaluate_window(col_array[r:r+WINDOW_LENGTH], piece)
    # Positive diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
    # Negative diagonal
    for r in range(ROW_COUNT-3):
        for c in range(COLUMN_COUNT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, piece)
    return score


### Valid Moves and Terminal State
- `get_valid_locations(board)` returns a list of columns where a piece can be dropped.
- `is_terminal_node(board)` checks if the game is over (win or draw).


In [11]:
def get_valid_locations(board):
    return [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]

def is_terminal_node(board):
    return winning_move(board, AI1_PIECE) or winning_move(board, AI2_PIECE) or len(get_valid_locations(board)) == 0


### Minimax AI with Alpha-Beta Pruning
- Recursive function to simulate future moves.
- `maximizingPlayer` chooses the best move for the AI.
- `minimizingPlayer` simulates opponent's response.


In [12]:
def minimax(board, depth, alpha, beta, maximizingPlayer, piece):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, piece):
                return (None, 1000000)
            elif winning_move(board, AI1_PIECE if piece == AI2_PIECE else AI2_PIECE):
                return (None, -1000000)
            else:
                return (None, 0)
        else:
            return (None, score_position(board, piece))

    if maximizingPlayer:
        value = -math.inf
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, False, piece)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value
    else:
        value = math.inf
        column = random.choice(valid_locations)
        opp_piece = AI1_PIECE if piece == AI2_PIECE else AI2_PIECE
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, opp_piece)
            new_score = minimax(b_copy, depth-1, alpha, beta, True, piece)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value


### Main Console Loop for AI vs AI Game
- Randomly choose which AI starts.
- Each AI uses `minimax` to choose the best move.
- Game continues until a win or draw.


In [13]:
board = create_board()
print_board(board)
game_over = False
turn = random.randint(AI1, AI2)

while not game_over:
    if turn == AI1:
        col, _ = minimax(board, 5, -math.inf, math.inf, True, AI1_PIECE)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI1_PIECE)
            print(f"AI 1 plays column {col}")
            print_board(board)
            if winning_move(board, AI1_PIECE):
                print("AI 1 wins!!")
                game_over = True
    else:
        col, _ = minimax(board, 5, -math.inf, math.inf, True, AI2_PIECE)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI2_PIECE)
            print(f"AI 2 plays column {col}")
            print_board(board)
            if winning_move(board, AI2_PIECE):
                print("AI 2 wins!!")
                game_over = True

    if len(get_valid_locations(board)) == 0 and not game_over:
        print("Draw!")
        game_over = True

    turn += 1
    turn %= 2


[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 1 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]]
AI 1 plays column 2
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
AI 2 plays column 3
[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 2. 0. 0. 0.]]
AI 1 plays column 2
[[0. 0. 0.

## <center> human VS AI